# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1f182a1c40>
├── 'a' --> tensor([[ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015]])
└── 'x' --> <FastTreeValue 0x7f1f182a1bb0>
    └── 'c' --> tensor([[-0.0599, -1.9261,  1.0801,  0.5003],
                        [ 1.8525,  1.5252,  1.0359,  0.1305],
                        [-1.5540,  0.4817,  0.2730,  0.6077]])

In [4]:
t.a

tensor([[ 1.0595, -0.8830,  1.6696],
        [-1.5925,  1.6795,  2.3015]])

In [5]:
%timeit t.a

65.2 ns ± 0.0615 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1f182a1c40>
├── 'a' --> tensor([[ 1.2599, -0.9822, -0.6727],
│                   [-0.3040, -0.8615,  0.5037]])
└── 'x' --> <FastTreeValue 0x7f1f182a1bb0>
    └── 'c' --> tensor([[-0.0599, -1.9261,  1.0801,  0.5003],
                        [ 1.8525,  1.5252,  1.0359,  0.1305],
                        [-1.5540,  0.4817,  0.2730,  0.6077]])

In [7]:
%timeit t.a = new_value

67.9 ns ± 0.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0595, -0.8830,  1.6696],
               [-1.5925,  1.6795,  2.3015]]),
    x: Batch(
           c: tensor([[-0.0599, -1.9261,  1.0801,  0.5003],
                      [ 1.8525,  1.5252,  1.0359,  0.1305],
                      [-1.5540,  0.4817,  0.2730,  0.6077]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0595, -0.8830,  1.6696],
        [-1.5925,  1.6795,  2.3015]])

In [11]:
%timeit b.a

60.8 ns ± 0.0148 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3610, -0.3330, -0.0552],
               [-0.3860,  0.0198,  1.1163]]),
    x: Batch(
           c: tensor([[-0.0599, -1.9261,  1.0801,  0.5003],
                      [ 1.8525,  1.5252,  1.0359,  0.1305],
                      [-1.5540,  0.4817,  0.2730,  0.6077]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 3.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

830 ns ± 0.71 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1f182a1910>
├── 'a' --> tensor([[[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]],
│           
│                   [[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]],
│           
│                   [[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]],
│           
│                   [[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]],
│           
│                   [[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]],
│           
│                   [[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]],
│           
│                   [[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]],
│           
│                   [[ 1.0595, -0.8830,  1.6696],
│                    [-1.5925,  1.6795,  2.3015]]])
└── 'x' --> <FastTreeValue 0x7f1f182a1700>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1e6bf99f40>
├── 'a' --> tensor([[ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015],
│                   [ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015],
│                   [ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015],
│                   [ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015],
│                   [ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015],
│                   [ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015],
│                   [ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015],
│                   [ 1.0595, -0.8830,  1.6696],
│                   [-1.5925,  1.6795,  2.3015]])
└── 'x' --> <FastTreeValue 0x7f1e72c55f70>
    └── 'c' --> tensor([[-0.0599, -1.9261,  1.0801,  0.5003],
                        [ 1.8525,  1.5252,  1.0359,  0.1305],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 86.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 43.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0599, -1.9261,  1.0801,  0.5003],
                       [ 1.8525,  1.5252,  1.0359,  0.1305],
                       [-1.5540,  0.4817,  0.2730,  0.6077]],
              
                      [[-0.0599, -1.9261,  1.0801,  0.5003],
                       [ 1.8525,  1.5252,  1.0359,  0.1305],
                       [-1.5540,  0.4817,  0.2730,  0.6077]],
              
                      [[-0.0599, -1.9261,  1.0801,  0.5003],
                       [ 1.8525,  1.5252,  1.0359,  0.1305],
                       [-1.5540,  0.4817,  0.2730,  0.6077]],
              
                      [[-0.0599, -1.9261,  1.0801,  0.5003],
                       [ 1.8525,  1.5252,  1.0359,  0.1305],
                       [-1.5540,  0.4817,  0.2730,  0.6077]],
              
                      [[-0.0599, -1.9261,  1.0801,  0.5003],
                       [ 1.8525,  1.5252,  1.0359,  0.1305],
                       [-1.5540,  0.4817,  0.2730,  0.6077]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0599, -1.9261,  1.0801,  0.5003],
                      [ 1.8525,  1.5252,  1.0359,  0.1305],
                      [-1.5540,  0.4817,  0.2730,  0.6077],
                      [-0.0599, -1.9261,  1.0801,  0.5003],
                      [ 1.8525,  1.5252,  1.0359,  0.1305],
                      [-1.5540,  0.4817,  0.2730,  0.6077],
                      [-0.0599, -1.9261,  1.0801,  0.5003],
                      [ 1.8525,  1.5252,  1.0359,  0.1305],
                      [-1.5540,  0.4817,  0.2730,  0.6077],
                      [-0.0599, -1.9261,  1.0801,  0.5003],
                      [ 1.8525,  1.5252,  1.0359,  0.1305],
                      [-1.5540,  0.4817,  0.2730,  0.6077],
                      [-0.0599, -1.9261,  1.0801,  0.5003],
                      [ 1.8525,  1.5252,  1.0359,  0.1305],
                      [-1.5540,  0.4817,  0.2730,  0.6077],
                      [-0.0599, -1.9261,  1.0801,  0.5003],
                   

In [28]:
%timeit Batch.cat(batches)

150 µs ± 2.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
